In [1]:
# Imports and Setup
import sys
from pathlib import Path
import pandas as pd

# Ensure we are operating relative to the /functions directory
FUNCTIONS_DIR = Path.cwd().parent
sys.path.insert(0, str(FUNCTIONS_DIR))
print(f'FUNCTIONS_DIR: {FUNCTIONS_DIR}')

# Firebase setup
from firebase_admin import initialize_app, credentials, get_app
from google.cloud import firestore as google_firestore
import logging

logging.basicConfig(level=logging.INFO)

# Use direct path to service account key inside /functions
service_account_path = FUNCTIONS_DIR / 'serviceAccountKey.json'

# Initialize Firebase Admin SDK
try:
    app = get_app()
except ValueError:
    cred = credentials.Certificate(service_account_path)
    app = initialize_app(cred, {'storageBucket': 'uno-y-b48fb.appspot.com'})

# Create Firestore client using the same credentials
firestore_client = google_firestore.Client.from_service_account_json(str(service_account_path))

# planogram imports
from planogram import data_loader

# Load data
print("Loading data...")
firebase_data = data_loader.load_firebase_collections(firestore_client)  # Loads all collections
sales_data = data_loader.load_all_sales_data(firestore_client, fetch_latest_sales=True, firebase_data=firebase_data) # Loads and combines historical + latest sales
sales_data = data_loader.enrich_with_purchase_prices(sales_data, firebase_data.get('product_purchase_prices'))
print("Data loaded.")

products_df = firebase_data.get('products')
machines_df = firebase_data.get('machines')

from planogram.config import PRODUCT_INFORMATION_PATH
product_information_df = pd.read_json(PRODUCT_INFORMATION_PATH)

FUNCTIONS_DIR: /Users/nainajnaho/PycharmProjects/SAM/functions


Loading data...
Loading collection: app_machines
app_machines slots cleaned: 8193 -> 8172 (removed 21, 0.26%)
app_machines: 234 rows (machines themselves not filtered)
Loading collection: products
products (filtered unspecific rows): 2524 -> 2019 rows (removed 505, 20.01%)
Loading collection: product_purchase_prices
product_purchase_prices: 1370 -> 1370 rows (removed 0, 0.00%)
Loading collection: product_nayax_mapping
product_nayax_mapping: 5317 -> 5317 rows (removed 0, 0.00%)
Loading historical sales from Parquet files...
Historical sales (after filtering): 6979402 -> 5390446 rows (removed 1588956, 22.77%)
Fetching latest sales data...
Fetching sales after: 2025-11-18 23:59:50+00:00
Enriching latest sales data...
Cached machines data missing 'address' column (likely app_machines). Fetching full machines data...
Fetching machines collection...


/Users/nainajnaho/PycharmProjects/SAM/functions/planogram/data_loader.py:345: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  enriched_df[col] = enriched_df[col].fillna(False).astype(bool)
/Users/nainajnaho/PycharmProjects/SAM/functions/planogram/data_loader.py:345: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  enriched_df[col] = enriched_df[col].fillna(False).astype(bool)


Data loaded.


# Train all

In [2]:
from planogram.train_all import train_all

artifacts = train_all(
    sales_df=sales_data,
    machines_df=machines_df,
    products_df=products_df,
    product_information_df=product_information_df,
)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: /Users/nainajnaho/PycharmProjects/SAM/functions/ragnar/models/all-MiniLM-L6-v2


Loading local SentenceTransformer model from /Users/nainajnaho/PycharmProjects/SAM/functions/ragnar/models/all-MiniLM-L6-v2


Batches:   0%|          | 0/64 [00:00<?, ?it/s]

INFO:planogram.train_all:Trained uniqueness_model
INFO:planogram.train_all:Trained location_mapping
INFO:planogram.healthiness_model:Starting healthiness model training with 1023 products
INFO:planogram.healthiness_model:Extracted nutritional values for 1023 products
INFO:planogram.healthiness_model:Calculated Nutri-Scores for 1023 products
INFO:planogram.healthiness_model:Created 1023 EAN to letter score mappings from calculated Nutri-Scores
INFO:planogram.healthiness_model:Created complete mapping for 1464 EANs from product_database_df
INFO:planogram.healthiness_model:  - 1021 EANs have letter scores
INFO:planogram.healthiness_model:  - 453 EANs have NaN (no letter score available)
INFO:planogram.healthiness_model:  - 555 products skipped (no valid EAN)
INFO:planogram.train_all:Trained healthiness_mapping


In [3]:
from planogram.artifact_utils import save_artifacts

# Save
saved_paths = save_artifacts(artifacts)

✓ Saved uniqueness_model to /Users/nainajnaho/PycharmProjects/SAM/functions/planogram/artifacts/uniqueness_model.npz
✓ Saved location_mapping to /Users/nainajnaho/PycharmProjects/SAM/functions/planogram/artifacts/location_mapping.parquet
✓ Saved healthiness_mapping to /Users/nainajnaho/PycharmProjects/SAM/functions/planogram/artifacts/healthiness_mapping.json


In [4]:
from planogram.artifact_utils import load_artifacts


artifacts_loaded = load_artifacts()



✗ Failed to load .gitkeep: file not found
✓ Loaded healthiness_mapping
✓ Loaded location_mapping
✓ Loaded uniqueness_model


# Predict Static df

In [5]:
from planogram.predict_static import predict_static

static_predictions, letter_grade_mapping = predict_static(
    sales_df=sales_data,
    machines_df=machines_df,
    product_df=products_df,
    location_mapping_df=artifacts_loaded['location_mapping'],
    healthiness_mapping=artifacts_loaded['healthiness_mapping'],
)

✓ Models loaded from /Users/nainajnaho/PycharmProjects/SAM/functions/planogram/moaaz_trend/artifact


INFO:planogram.moaaz_trend.predict:Filtering raw sales data: 2024-12-19 00:00:00+00:00 onwards


🚀 STARTING ETL PIPELINE
Input: 771,411 raw transactions
Input columns: 32 - address, card_brand, card_type, category, currency, customer_id, ean, is_ICA_refiller, latitude, local_timestamp, longitude, machine_eva_group, machine_group_tag, machine_id, machine_key, machine_model, machine_name, machine_sub_group, moms, nayax_name, position, price, product_name, provider, purchase_price_kr, pusher_friendly, refiller, sielaff_id, spiral, subcategory, too_unspecific, width
Date range: 2024-12-19 00:00:05+00:00 to 2025-12-18 00:00:00+00:00
Machines: 230
Products: 853

🔧 STEP 1: DATA CLEANING
🧹 CLEANING SALES DATA
Starting with 771,411 transactions
✓ Removed 40,798 rows with missing essential fields
✓ Removed 35 test transactions (price ≤ 2 SEK)
✓ Removed 3 transactions from 1 test machines
✓ Removed 5,603 rows with missing provider
✓ Removed 20,483 duplicate transactions
✓ Kept 14 relevant columns
✅ CLEANING COMPLETE
   Original: 771,411 transactions
   Removed:  66,922 transactions (8.7%)
  

INFO:planogram.moaaz_trend.utils.calendar:Loaded 180 Swedish holidays


✓ Removed 0 entries from 31 machines with decommissioned periods
✅ REMOVAL COMPLETE
   Original records: 281,237
   Removed:          33,584 (11.9%)
   Final records:    247,653
✅ MACHINE SNAPSHOTS COMPLETE
✓ After snapshot creation: 247,653 records
✓ Columns: 27 - _is_zero, _streak_group, category_first, customer_id_first, date_key, ean, first_sale_timestamp, machine_eva_group_first, machine_key, machine_sub_group_first, position, prev_ean, price_mean, product_name_first, provider_first, purchase_price_kr_first, refiller_first, subcategory_first, was_replaced, week, week_end, week_end_first, week_share, week_start, week_start_first, weekly_sales, year
✓ Total sales: 704,447 (expected: 704,489, diff: 42)

📅 STEP 3: CALENDAR INTEGRATION
📅 ADDING CALENDAR INFORMATION
Processing 247,653 weekly records
✓ Initializing Swedish holiday calendar
✓ Found 53 unique weeks
✓ Pre-calculating working days and holidays...
✓ Pre-calculated calendar info for 53 unique weeks
✓ Mapping calendar info (vec

INFO:planogram.moaaz_trend.predict:Preparing prediction data for target week: 2025-12-22 (derived from max history: 2025-12-15)
INFO:planogram.moaaz_trend.predict:Using 234 machines from machines_df
INFO:planogram.moaaz_trend.predict:Using 1464 products from product_df
INFO:planogram.moaaz_trend.predict:Created 342,576 (machine, product) combinations
INFO:planogram.moaaz_trend.predict:Created 342,576 future prediction rows for 2025-12-22



📊 Creating 7 feature group(s): BASE, TEMPORAL, PRODUCT, MACHINE, HISTORICAL_SALES, PRODUCT_LIFECYCLE, BRAND


Feature Groups: 100%|██████████| 7/7 [00:32<00:00,  4.61s/it]     
INFO:planogram.moaaz_trend.predict:Generating predictions for 342576 items for week of 2025-12-22



Generating Predictions: RECURSIVE MULTI
✓ Prepared 342,576 samples for prediction
✓ Generated 342,576 predictions



In [10]:
import pickle
import os

# Define Output Directory
OUTPUT_DIR = FUNCTIONS_DIR / 'data' / 'pipeline_artifacts'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"Saving artifacts to {OUTPUT_DIR}...")

# --- Data Cleaning for Parquet ---
# 1. Fix products_df 'pusher_friendly' (mixed bool/str)
if 'pusher_friendly' in products_df.columns:
    products_df['pusher_friendly'] = products_df['pusher_friendly'].replace({'True': True, 'False': False})
    products_df['pusher_friendly'] = products_df['pusher_friendly'].fillna(False).astype(bool)

# 2. Fix machines_df 'slots' (likely mixed types or large ints parsed as objects)
if 'slots' in machines_df.columns:
    machines_df['slots'] = machines_df['slots'].astype(str)

# 3. Fix machines_df 'machine_group_tag' (mixed lists and non-lists)
if 'machine_group_tag' in machines_df.columns:
    machines_df['machine_group_tag'] = machines_df['machine_group_tag'].astype(str)

# 4. Save DataFrames
products_df.to_parquet(OUTPUT_DIR / 'products.parquet')
machines_df.to_parquet(OUTPUT_DIR / 'machines.parquet')
static_predictions.to_parquet(OUTPUT_DIR / 'static_predictions.parquet')
letter_grade_mapping.to_parquet(OUTPUT_DIR / 'letter_grade_mapping.parquet')
print("Saved DataFrames: products, machines, static_predictions, letter_grade_mapping")

# 5. Save Artifacts Dictionary
artifacts_to_save = artifacts if 'artifacts' in locals() else artifacts_loaded
with open(OUTPUT_DIR / 'artifacts.pkl', 'wb') as f:
    pickle.dump(artifacts_to_save, f)
print("Saved artifacts.pkl")

Saving artifacts to /Users/nainajnaho/PycharmProjects/SAM/functions/data/pipeline_artifacts...
Saved DataFrames: products, machines, static_predictions, letter_grade_mapping
Saved artifacts.pkl


# New Machine sales forcasting

In [11]:
# 1. IMPORTS
import pandas as pd
from planogram.new_location_engine.LocationBasedForecasting.geo_similarity import calculate_similarity_to_multiple
from planogram.prediction_aggregator import aggregate_predictions_weighted

# Define Artifacts Directory
ARTIFACTS_DIR = FUNCTIONS_DIR / 'data' / 'pipeline_artifacts'

# Load standardized data from artifacts
print("Loading machines_df and static_predictions from artifacts...")
machines_df = pd.read_parquet(ARTIFACTS_DIR / 'machines.parquet')
static_predictions = pd.read_parquet(ARTIFACTS_DIR / 'static_predictions.parquet')

# Load other data (as per original logic)
ica_stores_df = data_loader.load_ica_stores() 
companies_df = data_loader.load_companies()

# 2. RUN SIMILARITY
all_similarities = calculate_similarity_to_multiple(
    target_lat=59.3293,
    target_lon=18.0686,
    reference_locations_df=machines_df,
    ica_stores_df=ica_stores_df,
    companies_df=companies_df,
    distance_weight=0.1,
    ica_weight=0.45,
    company_weight=0.45
)

# 3. FILTER
# Select only the 'machine_key' column for filtering
top_10_keys = all_similarities.head(10)['machine_key'] 
filtered_static_predictions = static_predictions[
    static_predictions['machine_key'].isin(top_10_keys)
]

# 4. AGGREGATE (Weighted Average)
aggregated_predictions = aggregate_predictions_weighted(
    predictions_df=filtered_static_predictions,
    similarity_df=all_similarities,
    product_col='product_name',
    machine_key_col='machine_key',
    similarity_col='similarity'
)

Loading machines_df and static_predictions from artifacts...
Loading ICA stores from /Users/nainajnaho/PycharmProjects/SAM/functions/data/processed/scb_supermarkets.parquet
Loading Companies from /Users/nainajnaho/PycharmProjects/SAM/functions/data/processed/scb_companies.parquet


In [12]:
aggregated_predictions

,product_name,machine_key,ean,week_start,confidence_score,machine_eva_group,location_fit_score,healthiness_letter,pred_week_1,pred_week_2,pred_week_3,pred_week_4
1097,Propud Bar Delicatoboll,Mat - Adapteo gamla_183104077,7350068295577.0,2025-12-22,0.494231,WORK,0.395000,D,3.734944,5.417837,6.637801,7.374113
1140,RFSU Ägglossningstest,Mat - Adapteo gamla_183104077,7310572100071.0,2025-12-22,0.000000,WORK,0.450810,None,3.718308,5.201966,7.170184,9.074837
42,Arla Proteindryck Chocolate 482ml,Mat - Adapteo gamla_183104077,5711953068874.0,2025-12-22,0.950000,WORK,0.626000,None,3.649868,5.611927,7.763406,9.328060
676,IQ Fuel Focus Grapefruit 33cl,Mat - Adapteo gamla_183104077,7350112800016.0,2025-12-22,0.000000,WORK,0.546732,C,3.566237,5.189045,6.368286,7.125912
1196,Risenta Sesamkaka,Mat - Adapteo gamla_183104077,7311312007421.0,2025-12-22,0.000000,WORK,0.075000,D,3.549701,5.337248,7.228704,8.671922
...,...,...,...,...,...,...,...,...,...,...,...,...
1277,Star Bar Salted Caramel,Mat - Adapteo gamla_183104077,7340028812791.0,2025-12-22,0.000000,WORK,0.500015,C,2.312937,4.286244,6.061988,7.142816
1275,Star Bar Coconut Dream,Mat - Adapteo gamla_183104077,7340028812784.0,2025-12-22,0.000000,WORK,0.500015,C,2.311120,3.747690,5.498010,6.912307
34,Ar Functional Upgrade 50cl,Mat - Adapteo gamla_183104077,7350117730202.0,2025-12-22,0.000000,WORK,0.546732,None,2.261405,4.670228,7.181346,9.276743
727,Kin Food Ryggbiff Grönsaker,Mat - Adapteo gamla_183104077,7350126470069.0,2025-12-22,1.000000,WORK,0.859000,B,2.242938,3.359365,5.024182,4.621859


In [ ]:
# Chose one machine
machine_df = machines_df[machines_df['machine_key'] == 'PRO M - Wämö_594015928']
machine_df

# FORTSÄTT HÄRIFRÅN

In [ ]:
static_predictions_machine = (
    static_predictions[static_predictions['machine_key'] == 'PRO M - Wämö_594015928']
)
len(static_predictions_machine)

In [ ]:
from planogram.predict_dynamic_autofill import predict_dynamic_autofill 

dynamic_predictions = predict_dynamic_autofill(
    machine_df=machine_df,
    product_df=products_df,
    uniqueness_model=artifacts_loaded['uniqueness_model'],
    cousin_model=artifacts_loaded['cousin_model'],
    static_predictions_df=static_predictions_machine,
    products_df=products_df,
)

In [ ]:
dynamic_predictions